In [22]:
# imports
import pandas as pd 
import numpy as np
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [23]:
# read in data
model_data = pd.read_csv('data/model_data.csv', index_col = 0)
y = model_data[['ARRIVAL_DELAY']]
x = model_data.drop('ARRIVAL_DELAY', axis = 1)
print x.shape
print y.shape
print list(x.columns)
model_data = model_data.reset_index(drop=True)
model_data.head()


(88500, 18)
(88500, 1)
['DAY', 'MONTH', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'TAIL_NUMBER', 'TEMPERATURE', 'ICON', 'HUMIDITY', 'SUMMARY', 'WIND_SPEED', 'VISIBILITY', 'PRECIP_INTENSITY', 'PRECIP_PROB', 'MODEL', 'YEAR']


,ARRIVAL_DELAY,DAY,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAIL_NUMBER,TEMPERATURE,ICON,HUMIDITY,SUMMARY,WIND_SPEED,VISIBILITY,PRECIP_INTENSITY,PRECIP_PROB,MODEL,YEAR
0,0,28,7,2,AA,ORD,CLT,7,N939UW,3,clear-day,3,Clear,6,5,1,1,757-2B7,1994
1,0,10,7,5,EV,ORD,MEM,12,N12552,2,partly-cloudy-day,2,Partly Cloudy,3,5,1,1,EMB-145LR,2002
2,0,25,9,5,UA,ORD,RSW,7,N26232,2,clear-day,4,Clear,6,4,1,1,737-824,1999
3,0,25,5,1,OO,ORD,BOI,12,N124SY,0,wind,3,Breezy and Overcast,8,4,1,4,ERJ 170-200 LR,2014
4,0,16,3,1,UA,ORD,SMF,19,N36447,3,clear-night,2,Clear,7,5,1,1,737-924ER,2012


In [24]:
delay_col = model_data.columns.get_loc('ARRIVAL_DELAY')
print delay_col
for i, row in model_data.iterrows():
    if i % 5000 == 0:
        print ('done ' + str(i))
    if model_data.iat[i, delay_col] == 1:
        model_data.iat[i,delay_col] = 0
    if model_data.iat[i,delay_col] == 2:
        model_data.iat[i,delay_col] = 1

0
done 0
done 5000
done 10000
done 15000
done 20000
done 25000
done 30000
done 35000
done 40000
done 45000
done 50000
done 55000
done 60000
done 65000
done 70000
done 75000
done 80000
done 85000


In [25]:
not_late = model_data[model_data['ARRIVAL_DELAY'] == 0]
print not_late.shape
late = model_data[model_data['ARRIVAL_DELAY'] == 1]
print late.shape

(59000, 19)
(29500, 19)


In [26]:
not_late = not_late.sample(29500)
print not_late.shape

(29500, 19)


In [27]:
model_data = pd.concat([not_late, late])
print model_data.shape
model_data.head()

(59000, 19)


,ARRIVAL_DELAY,DAY,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAIL_NUMBER,TEMPERATURE,ICON,HUMIDITY,SUMMARY,WIND_SPEED,VISIBILITY,PRECIP_INTENSITY,PRECIP_PROB,MODEL,YEAR
25767,0,15,12,2,UA,ORD,LAX,12,N14214,0,cloudy,3,Overcast,6,5,1,1,737-824,1998
18171,0,16,3,1,UA,ORD,MSP,15,N485UA,0,clear-day,1,Clear,7,5,1,1,A320-232,2001
30551,0,16,9,3,UA,ORD,IAH,12,N37468,4,wind,2,Breezy,5,5,1,1,737-924ER,2013
33469,0,22,4,3,EV,ORD,GSP,7,N14117,3,wind,2,Breezy,7,5,1,1,EMB-145XR,2002
47810,0,1,4,3,EV,ORD,LNK,19,N12552,2,wind,1,Breezy,7,5,1,1,EMB-145LR,2002


In [28]:
y = model_data[['ARRIVAL_DELAY']]
x = model_data.drop('ARRIVAL_DELAY', axis = 1)
print x.shape
print y.shape
print list(x.columns)
x.head()

(59000, 18)
(59000, 1)
['DAY', 'MONTH', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'TAIL_NUMBER', 'TEMPERATURE', 'ICON', 'HUMIDITY', 'SUMMARY', 'WIND_SPEED', 'VISIBILITY', 'PRECIP_INTENSITY', 'PRECIP_PROB', 'MODEL', 'YEAR']


,DAY,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAIL_NUMBER,TEMPERATURE,ICON,HUMIDITY,SUMMARY,WIND_SPEED,VISIBILITY,PRECIP_INTENSITY,PRECIP_PROB,MODEL,YEAR
25767,15,12,2,UA,ORD,LAX,12,N14214,0,cloudy,3,Overcast,6,5,1,1,737-824,1998
18171,16,3,1,UA,ORD,MSP,15,N485UA,0,clear-day,1,Clear,7,5,1,1,A320-232,2001
30551,16,9,3,UA,ORD,IAH,12,N37468,4,wind,2,Breezy,5,5,1,1,737-924ER,2013
33469,22,4,3,EV,ORD,GSP,7,N14117,3,wind,2,Breezy,7,5,1,1,EMB-145XR,2002
47810,1,4,3,EV,ORD,LNK,19,N12552,2,wind,1,Breezy,7,5,1,1,EMB-145LR,2002


In [29]:
# select features for SVM
svm_features = x[['AIRLINE','MONTH', 'DAY_OF_WEEK', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE', 'MODEL', 'YEAR', 'TEMPERATURE', 'WIND_SPEED', 'VISIBILITY', 'PRECIP_PROB', 'PRECIP_INTENSITY', 'HUMIDITY']]
print svm_features.shape

# encode non-numeric features
svm_features = pd.concat([svm_features.drop('AIRLINE', axis = 1), pd.get_dummies(svm_features[['AIRLINE']])], axis = 1)
svm_features = pd.concat([svm_features, pd.get_dummies(svm_features['MODEL'])], axis = 1)
svm_features = svm_features.drop('MODEL', axis = 1)
svm_features = pd.concat([svm_features, pd.get_dummies(svm_features['DESTINATION_AIRPORT'])], axis = 1)
svm_features = svm_features.drop('DESTINATION_AIRPORT', axis = 1)
svm_features = pd.concat([svm_features, pd.get_dummies(svm_features['YEAR'])], axis = 1)
svm_features = svm_features.drop('YEAR', axis = 1)
svm_features = svm_features.fillna(0)
print svm_features.shape

(59000, 13)
(59000, 251)


In [30]:
train_x, test_x, train_y, test_y = train_test_split(svm_features, y, train_size=0.8)
print train_y.shape
print train_x.shape

(47200, 1)
(47200, 251)


In [31]:
#svm with sampled data
svm_classifier = SVC()
svm_classifier.fit(train_x, np.ravel(train_y, order = 'C'))
svm_train_acc = metrics.accuracy_score(train_y, svm_classifier.predict(train_x))
svm_test_acc = metrics.accuracy_score(test_y, svm_classifier.predict(test_x))

# results
print ('svm train acc: ' + str(svm_train_acc))
print ('svm test acc: ' + str(svm_test_acc))
print ('svm precision: ' + str(metrics.precision_score(test_y, svm_classifier.predict(test_x), average = 'binary')))
print ('svm recall: ' + str(metrics.recall_score(test_y, svm_classifier.predict(test_x), average = 'binary')))

svm train acc: 0.618495762712
svm test acc: 0.610423728814
svm precision: 0.633825944171
svm recall: 0.523301135401


In [16]:
# decision tree with sampled data
dtc = tree.DecisionTreeClassifier()
dtc.fit(train_x, train_y)
dtc_train_acc = metrics.accuracy_score(train_y, dtc.predict(train_x))
dtc_test_acc = metrics.accuracy_score(test_y, dtc.predict(test_x))

# results
print ('decision tree train acc: ' + str(dtc_train_acc))
print ('decision tree test acc: ' + str(dtc_test_acc))
print ('decision tree precision: ' + str(metrics.precision_score(test_y, dtc.predict(test_x), average = 'binary')))
print ('decision tree recall: ' + str(metrics.recall_score(test_y, dtc.predict(test_x), average = 'binary')))

decision tree train acc: 0.999703389831
decision tree test acc: 0.598389830508
decision tree precision: 0.603462909523
decision tree recall: 0.59667673716


In [17]:
# random forest w/ sampled data
rfc = RandomForestClassifier(n_jobs = 2)
rfc.fit(train_x, train_y)
rfc_train_acc = metrics.accuracy_score(train_y, rfc.predict(train_x))
rfc_test_acc = metrics.accuracy_score(test_y, rfc.predict(test_x))
pred_y = rfc.predict(test_x)

# results
print ('random forest train acc: ' + str(rfc_train_acc))
print ('random forest test acc: ' + str(rfc_test_acc))
print ('random forest precision: ' + str(metrics.precision_score(test_y, rfc.predict(test_x), average = 'binary')))
print ('random forest recall: ' + str(metrics.recall_score(test_y, rfc.predict(test_x), average = 'binary')))

/Users/nnbenavides/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


random forest train acc: 0.983283898305
random forest test acc: 0.609152542373
random forest precision: 0.639916839917
random forest recall: 0.516616314199


In [13]:
cm = confusion_matrix(test_y, pred_y)
sns.set(font_scale=1.5)
sns.set_style(style='white')
cmap = sns.cubehelix_palette(as_cmap=True)
ax = sns.heatmap(cm, cmap=cmap)
ax.set(xlabel='Predicted Delay', ylabel='Actual Delay')
plt.xticks(rotation=25)
plt.tight_layout()
plt.savefig('CM_binarized.png')